## double dense

In [ ]:
import quartz
from quartz import layers
import numpy as np
import torch
import torch.nn as nn

input_dims = (1,1,1,1,)
l1_output_dim = 1
l2_output_dim = 1

t_max = 2**7
np.random.seed(seed=45)
weights1 = np.ones((l1_output_dim, np.product(input_dims[1:]))) / 2 # (np.random.rand(l1_output_dim, np.product(input_dims[1:])) - 0.5) # 
biases1 = np.zeros((l1_output_dim)) # (np.random.rand(l1_output_dim) - 0.5) / 2
weights2 = np.ones((l2_output_dim, l1_output_dim)) / 2 # (np.random.rand(l2_output_dim, l1_output_dim) - 0.5) # 
biases2 = np.zeros((l2_output_dim)) # (np.random.rand(l2_output_dim) - 0.5) / 2
inputs = np.ones((input_dims)) * 1 # np.random.rand(np.product(input_dims)) # np.zeros((input_dims)) / 2 # 

loihi_model = quartz.Network(t_max, [
    layers.InputLayer(dims=input_dims[1:]),
    layers.Dense(weights=weights1, biases=None, weight_acc=128),
    layers.Dense(weights=weights2, biases=None, weight_acc=128),
    layers.Dense(weights=weights2, biases=None, weight_acc=128),
])

relcos1 = [quartz.probe(neuron) for neuron in loihi_model.layers[1].neurons() if 'relco' in neuron.name]
relcos2 = [quartz.probe(neuron) for neuron in loihi_model.layers[2].neurons() if 'relco' in neuron.name]
rect1 = [quartz.probe(neuron) for neuron in loihi_model.layers[1].neurons() if 'rect' in neuron.name]
rect2 = [quartz.probe(neuron) for neuron in loihi_model.layers[2].neurons() if 'rect' in neuron.name]

model = nn.Sequential(
    nn.Linear(in_features=np.product(input_dims[1:]), out_features=l1_output_dim), nn.ReLU(),
    nn.Linear(in_features=l1_output_dim, out_features=l2_output_dim), nn.ReLU(),
)
q_weights1, q_biases1, q_inputs = quartz.utils.quantize_values(weights1, biases1, inputs, loihi_model.layers[1].weight_acc, t_max)
q_weights2, q_biases2, q_inputs = quartz.utils.quantize_values(weights2, biases2, inputs, loihi_model.layers[2].weight_acc, t_max)
model[0].weight = nn.Parameter(torch.tensor(q_weights1))
model[0].bias = nn.Parameter(torch.tensor(q_biases1))
model[2].weight = nn.Parameter(torch.tensor(q_weights2))
model[2].bias = nn.Parameter(torch.tensor(q_biases2))
model_output = model(torch.tensor(q_inputs).reshape(input_dims[0],-1)).detach().numpy()
model_output

In [ ]:
loihi_model(inputs)

In [ ]:
relcos1[0].plot()

In [ ]:
relcos2[0].plot()

In [ ]:
rect1[0].plot()

In [ ]:
ok = [relco.plot() for relco in relcos1]

## double conv

In [ ]:
import quartz
from quartz import layers
import numpy as np
import torch
import torch.nn as nn

input_dims = (1,1,10,10)
conv_weight_dims1 = (6,1,8,8)
conv_weight_dims2 = (10,6,3,3)

t_max = 2**8
conv_kernel_size1 = conv_weight_dims1[2:]
conv_kernel_size2 = conv_weight_dims2[2:]
conv_out_dim1 = conv_weight_dims1[0]
conv_out_dim2 = conv_weight_dims2[0]

weights1 = (np.random.rand(*conv_weight_dims1)-0.5) / 2
biases1 = (np.random.rand(conv_out_dim1)-0.5) / 2
weights2 = (np.random.rand(*conv_weight_dims2)-0.5) / 2
biases2 = (np.random.rand(conv_out_dim2)-0.5) / 2
inputs = np.random.rand(*input_dims) / 3

loihi_model = quartz.Network([
    layers.InputLayer(dims=input_dims[1:]),
    layers.Conv2D(weights=weights1, biases=biases1),
    layers.Conv2D(weights=weights2, biases=biases2),
])

relcos1 = [quartz.probe(block) for block in loihi_model.layers[1].blocks if 'relco' in block.name]
relcos2 = [quartz.probe(block) for block in loihi_model.layers[2].blocks if 'relco' in block.name]

model = nn.Sequential(
    nn.Conv2d(in_channels=conv_weight_dims1[1], out_channels=conv_weight_dims1[0], kernel_size=conv_kernel_size1), nn.ReLU(),
    nn.Conv2d(in_channels=conv_weight_dims2[1], out_channels=conv_weight_dims2[0], kernel_size=conv_kernel_size2), nn.ReLU(),
)

q_weights1, q_biases1, q_inputs = quartz.utils.quantize_values(weights1, biases1, inputs, loihi_model.layers[1].weight_acc, t_max)
q_weights2, q_biases2, q_inputs = quartz.utils.quantize_values(weights2, biases2, inputs, loihi_model.layers[2].weight_acc, t_max)
model[0].weight = nn.Parameter(torch.tensor(q_weights1))
model[0].bias = nn.Parameter(torch.tensor(q_biases1))
model[2].weight = nn.Parameter(torch.tensor(q_weights2))
model[2].bias = nn.Parameter(torch.tensor(q_biases2))
model_output = model(torch.tensor(q_inputs)).detach().numpy()
model_output

In [ ]:
loihi_model(inputs, t_max)

In [ ]:
relcos1[8].plot()

In [ ]:
relcos2[8].plot()

## double convpool

In [ ]:
import quartz
from quartz import layers
import numpy as np
import torch
import torch.nn as nn

input_dims = (1,1,10,10)
weight_dims = ( 3,1,3,3)
weight_dims2 = (5,3,3,3)
t_max = 2**8
conv_kernel_size = weight_dims[2:]
pooling_kernel_size = [2,2]
pooling_stride = 2

np.random.seed(seed=44)
np.set_printoptions(suppress=True)
weights1 = (np.random.rand(*weight_dims)-0.5) / 5 # np.zeros(weight_dims) #
weights2 = (np.random.rand(*weight_dims2)-0.5) / 5# np.zeros(weight_dims) #
biases1 = (np.random.rand(weight_dims[0])-0.5) / 3 # np.zeros((weight_dims[0])) # 
biases2 = (np.random.rand(weight_dims2[0])-0.5) / 3 # np.zeros((weight_dims2[0])) # 
inputs = np.random.rand(*input_dims) / 3

loihi_model = quartz.Network([
    layers.InputLayer(dims=input_dims[1:]),
    layers.Conv2D(weights=weights1, biases=biases1),
    layers.MaxPool2D(kernel_size=pooling_kernel_size),
    layers.Conv2D(weights=weights2, biases=biases2),
    layers.MaxPool2D(kernel_size=pooling_kernel_size),
])
    
relcos1 = [quartz.probe(block) for block in loihi_model.layers[1].blocks if 'relco' in block.name]
relcos2 = [quartz.probe(block) for block in loihi_model.layers[3].blocks if 'relco' in block.name]
wtas1 = [quartz.probe(block) for block in loihi_model.layers[2].blocks if 'wta' in block.name]
wtas2 = [quartz.probe(block) for block in loihi_model.layers[4].blocks if 'wta' in block.name]

model = nn.Sequential(
    nn.Conv2d(in_channels=weight_dims[1], out_channels=weight_dims[0], kernel_size=conv_kernel_size), nn.ReLU(),
    nn.MaxPool2d(kernel_size=pooling_kernel_size, stride=pooling_stride), nn.ReLU(),
    nn.Conv2d(in_channels=weight_dims2[1], out_channels=weight_dims2[0], kernel_size=conv_kernel_size), nn.ReLU(),
    nn.MaxPool2d(kernel_size=pooling_kernel_size, stride=pooling_stride), nn.ReLU(),
)
q_weights1, q_biases1, q_inputs = quartz.utils.quantize_values(weights1, biases1, inputs, loihi_model.layers[1].weight_acc, t_max)
q_weights2, q_biases2, q_inputs = quartz.utils.quantize_values(weights2, biases2, inputs, loihi_model.layers[2].weight_acc, t_max)
model[0].weight = nn.Parameter(torch.tensor(q_weights1))
model[0].bias = nn.Parameter(torch.tensor(q_biases1))
model[4].weight = nn.Parameter(torch.tensor(q_weights2))
model[4].bias = nn.Parameter(torch.tensor(q_biases2))
model_output = model(torch.tensor(q_inputs)).detach().numpy()
model_output

In [ ]:
loihi_model(inputs, t_max)

In [ ]:
ok = [relco.plot() for relco in relcos2]

In [ ]:
ok = [wta.plot() for wta in wtas2]

## convpool - conv

In [ ]:
import quartz
from quartz import layers
import numpy as np
import torch
import torch.nn as nn

# input_dims = ( 5,14,14)
# weight_dims = ( 8,5,5,5)
# weight_dims2 = (120,8,5,5)
input_dims = ( 1,7,7)
weight_dims = ( 2,1,2,2)
weight_dims2 = (4,2,3,3)

# input_dims = ( 1,6,6)
# weight_dims = ( 2,1,3,3)
# weight_dims2 = (3,2,2,2)

t_max = 2**9
weight_acc = 2**7
conv_kernel_size = weight_dims[2:]
pooling_kernel_size = [2,2]
pooling_stride = 2

np.random.seed(seed=34)
np.set_printoptions(suppress=True)
weights = (np.random.rand(*weight_dims)-0.5) / 4 # np.zeros(weight_dims) #
weights2 = (np.random.rand(*weight_dims2)-0.5) / 4 # np.zeros(weight_dims) #
biases = (np.random.rand(weight_dims[0])-0.5) / 2
biases2 = (np.random.rand(weight_dims2[0])-0.5) / 2

biases = np.zeros(weight_dims[0]) # (np.random.rand(weight_dims[0])-0.5) / 2
biases2 = np.zeros(weight_dims2[0])

loihi_model = quartz.Network([
    layers.InputLayer(dims=input_dims),
    layers.ConvPool2D(weights=weights, biases=None, pool_kernel_size=pooling_kernel_size, pool_stride=pooling_stride, weight_acc=weight_acc),
    layers.Conv2D(weights=weights2, biases=None, weight_acc=weight_acc),
    layers.MonitorLayer(),
])

trigger0 = quartz.probe(loihi_model.layers[0].blocks[-1])
trigger1 = quartz.probe(loihi_model.layers[1].blocks[0])
probes = [quartz.probe(block) for block in loihi_model.layers[1].output_blocks()]

values = np.random.rand(np.product(input_dims)) / 2
inputs = quartz.utils.decode_values_into_spike_input(values, t_max)

model = nn.Sequential(
    nn.Conv2d(in_channels=weight_dims[1], out_channels=weight_dims[0], kernel_size=conv_kernel_size), nn.ReLU(),
    nn.MaxPool2d(kernel_size=pooling_kernel_size, stride=pooling_stride), nn.ReLU(),
    nn.Conv2d(in_channels=weight_dims2[1], out_channels=weight_dims2[0], kernel_size=conv_kernel_size), nn.ReLU(),
)
model[0].weight = nn.Parameter(torch.tensor(weights))
model[0].bias = nn.Parameter(torch.tensor((biases)))
model[4].weight = nn.Parameter(torch.tensor(weights2))
model[4].bias = nn.Parameter(torch.tensor((biases2)))
model_output = model(torch.tensor(values.reshape(1, *input_dims[:3]))).squeeze().detach().numpy()
model_output

In [ ]:
loihi_model(inputs, t_max).reshape(model_output.shape)

In [ ]:
loihi_model1(inputs, t_max).reshape(model_output.shape)